# RAGAS를 활용한 데이터 생성

RAGAS 라이브러리에서는 데이터 생성 및 합성 기능 또한 제공합니다.

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

OpenAI API Key: ········


In [3]:
from ragas.testset import TestsetGenerator
from ragas.testset.graph import KnowledgeGraph, Node, NodeType
from ragas.testset.transforms import apply_transforms, HeadlinesExtractor, HeadlineSplitter, KeyphrasesExtractor
from ragas.testset.persona import Persona
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 1. LLM과 임베딩 모델 설정 및 Ragas 래퍼 적용
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# embedding_model
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

# 2. 문서 로드 및 전처리
loader = PyPDFLoader("./National_AI_Plan.pdf")
pdf = loader.load()


# 문서 분할 (긴 문서의 경우)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 튜토리얼의 HeadlineSplitter max_tokens와 유사하게 설정
    chunk_overlap=200
)
documents = text_splitter.split_documents(pdf)

# 너무 짧은 문서 제거 (Ragas 처리 시 오류 방지)
processed_documents = [doc for doc in documents if len(doc.page_content) > 100]
print(f"총 {len(pdf)} 페이지에서 {len(processed_documents)}개의 문서 조각을 생성했습니다.")

비정형 데이터(문서 텍스트)를 Ragas가 이해하고 활용할 수 있는 구조적인 형태로 만들기 위해 지식 그래프로 변환합니다.  

`TestsetGenerator`는 전체 문서에서 무작위로 질문을 만드는 것이 아니라, 지식 그래프의 특정 노드 하나를 선택하고 그 노드의 내용과 속성(헤드라인, 키워드 등)에 집중하여 질문을 생성합니다. 이는 생성된 질문이 특정 컨텍스트에 기반하고, 그에 대한 '정답' 역시 해당 노드 내용으로 한정될 수 있게 해줍니다. 이는 RAG 평가의 신뢰도를 높입니다.

In [ ]:
# 3. 문서를 지식 그래프의 노드로 변환
kg = KnowledgeGraph()

for doc in processed_documents:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

print(f"\n초기 지식 그래프 상태: {kg}")

지식 그래프의 노드에 저장된 정보를 더 풍부하게 만들어, 더 다양하고 품질 좋은 질문을 생성하기 위해서 변환을 적용합니다.

- `HeadlinesExtractor` (헤드라인 추출): 문서 조각에서 소제목이나 핵심 주제를 추출하여 노드에 'headlines' 속성으로 추가합니다. 

- `KeyphrasesExtractor` (키워드 추출): 문서 조각의 핵심 키워드들을 추출하여 노드에 'keyphrases' 속성으로 추가합니다. 

- `HeadlineSplitter` (헤드라인 기준 분할): 하나의 노드가 너무 길고 여러 주제를 포함하고 있을 때, 추출된 헤드라인을 기준으로 노드를 더 작고 의미 있는 단위로 쪼개줍니다. 이렇게 하면 각 노드가 하나의 주제에 더 집중하게 되어, 질문과 정답의 명확성이 높아집니다.

In [ ]:
# 4. 변환(Transforms) 설정 및 적용
# 헤드라인, 키워드 추출 등을 통해 그래프를 풍부하게 만듭니다.
headline_extractor = HeadlinesExtractor(llm=generator_llm, max_num=5) # 문서당 최대 5개 헤드라인 추출
headline_splitter = HeadlineSplitter(max_tokens=1500)
keyphrase_extractor = KeyphrasesExtractor(llm=generator_llm)

transforms = [
    headline_extractor,
    headline_splitter,
    keyphrase_extractor
]

# 변환 적용
apply_transforms(kg, transforms=transforms)
print(f"변환 적용 후 지식 그래프 상태: {kg}\n")

In [ ]:
# 5. 페르소나 설정
# '국가 AI 전략' 문서에 어울리는 가상의 페르소나를 정의합니다.
persona_policy_maker = Persona(
    name="정부 정책 입안자",
    role_description="국가 AI 전략의 실행 계획, 예산, 부처 간 협력 방안에 대해 구체적이고 현실적인 질문을 합니다. 정책의 기대 효과와 잠재적 리스크에 관심이 많습니다.",
)

persona_ai_researcher = Persona(
    name="AI 연구원",
    role_description="AI 기술 개발, 연구 지원, 인재 양성, 데이터 인프라 구축 등 기술적인 세부 사항에 대해 깊이 있는 질문을 합니다. 최신 기술 동향과 전략의 연관성을 파악하고자 합니다.",
)

persona_concerned_citizen = Persona(
    name="관심 있는 시민",
    role_description="AI 기술이 일상 생활과 사회에 미칠 영향, 특히 윤리 문제, 일자리 변화, 개인정보 보호에 대해 우려 섞인 질문을 합니다. 일반인이 이해하기 쉬운 설명을 요구합니다.",
)

personas = [persona_policy_maker, persona_ai_researcher, persona_concerned_citizen]


- `Synthesizer`: 질문과 답변 쌍을 생성하는 엔진입니다. 지식 그래프의 노드와 페르소나 정보를 입력받아, LLM을 활용하여 최종적인 테스트 데이터를 만들어내는 역할을 합니다.

- `SingleHopSpecificQuerySynthesizer`: 하나의 노드에서 얻은 구체적인 정보(헤드라인, 키워드 등)를 바탕으로, 해당 노드의 내용만으로 답변할 수 있는 명확한 질문과 정답 쌍을 만드는 역할을 합니다.
    - `SingleHop`: 질문에 대한 답을 하나의 문서(또는 하나의 노드) 안에서 찾을 수 있다는 의미
    - `MultiHop`: 여러 문서에 흩어져 있는 정보를 종합해야만 답할 수 있는 복잡한 질문
    - `SpecificQuery`: 일반적이거나 모호한 질문이 아닌, 구체적인 정보를 묻는 질문을 생성

In [ ]:
# 6. 신디사이저(Synthesizer)를 이용한 질문 생성 방식 정의
# 헤드라인과 키워드를 기반으로 질문을 생성하도록 설정합니다(1:1 비율).
query_distribution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=generator_llm, property_name="headlines"),
        0.5,
    ),
    (
        SingleHopSpecificQuerySynthesizer(
            llm=generator_llm, property_name="keyphrases"
        ),
        0.5,
    ),
]

In [ ]:
# 7. 테스트셋 생성기(TestsetGenerator) 초기화 및 생성
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    knowledge_graph=kg,
    persona_list=personas,
)

# 테스트셋 생성 실행 (시간이 다소 소요될 수 있습니다)
testset_size = 10
testset = generator.generate(testset_size=testset_size, query_distribution=query_distribution)


In [ ]:
# 8. 생성된 테스트셋을 pandas DataFrame으로 변환
df = testset.to_pandas()
pd.set_option('display.max_colwidth', None)
print(df)

In [ ]:
df.to_csv("ragas_testset.csv", index=False)